<a href="https://colab.research.google.com/github/SpencerPao/Natural-Language-Processing/blob/main/Question%20Answering%20Modeling/Question_Answering_Modeling_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import logging
import torch
from transformers import RobertaForQuestionAnswering, RobertaTokenizer
from transformers import Trainer, TrainingArguments
import pandas as pd
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
import numpy as np
# """https://huggingface.co/deepset/roberta-base-squad2"""
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "nur-dev/roberta-kaz-large"


logging.basicConfig(level=logging.DEBUG)
tokenizer = AutoTokenizer.from_pretrained("nur-dev/roberta-kaz-large")
model = AutoModelForMaskedLM.from_pretrained("nur-dev/roberta-kaz-large")

class CustomQuestionAnsweringModel(RobertaForQuestionAnswering):
    def __init__(self, masked_model):
        super().__init__(masked_model.config)
        self.roberta.load_state_dict(masked_model.roberta.state_dict())

qa_model = CustomQuestionAnsweringModel(model)

# a) Get predictions
nlp = pipeline('question-answering', model=qa_model, tokenizer=model_name)
QA_input = {
    'question': 'Энергетикалық алмасу дегеніміз не?',
    'context': 'АТФ - ағзалардың бүкіл пайдалы энергиясы іс жүзінде сол арқылы «өтетін» зат. Бүкіл тіршілік үдерістері негізіне жататын химиялық реакциялардың жасушада жүзеге асуына жылу, механикалық, өзге де ешқандай энергия түрі пайдаланылмайды. АТФ-ны «жасушалардың энергетикалық нарқы» деп жиі айтатыны сондықтанболса керек. Заттар ыдырау арқылы энергиясын АТФ синтезіне береді. Осыған қатарласа ағзамен және жарғақша арқылы биосинтездің, өсудің, бұлшықет жиырылуының, жүйке серпілісінің заттар тасымалдауды және өзге де көптеген тіршілік үдерістері жүреді. Осы үдерістердің барлығына тек АТФ-ның ыдырау энергиясы ғана шығындалады. АТФ-ның бір молекуласы тәулігіне 2-ден 3 мың рет ыдырап, қосылыса алады!'
}
res = nlp(QA_input)


In [ ]:
res

# RoBERTa Architecture
if you are curious...

In [3]:
model = qa_model
model

CustomQuestionAnsweringModel(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

# How to fine-tune a QA Model?
- Definitley need a GPU. Else, you are looking at a fine-tuning phase that is at least 10 times slower to train.
- Let's leverage an already built training script.
    - [Located here](https://github.com/huggingface/transformers/blob/b90745c5901809faef3136ed09a689e7d733526c/examples/run_squad.py); The execution is in the cell below.
    
What is this script doing?
- Gets whichever pretrained model you want to use (we are using RoBERTa in this case, but you can use a different pretrained model)
- Input dataset is converted into features
    - The featured dataset is saved in cache, so you don't have to necessarily rerun this process once more for this model.
- Ensure that the <b> --do_train </b> is enabled; This commences the training.
- When training is done, the outputs of the model are saved in a <b> output_dir / checkpoint - step_number</b>

In [43]:
"""You could go through this route and run your training script here via command line with the associated parameters
and the script."""
## More information on how to do this can be found here: https://qa.fastforwardlabs.com/pytorch/hugging%20face/wikipedia/bert/transformers/2020/05/19/Getting_Started_with_QA.html
# Download training script from the transformers library
# !curl -L -O https://github.com/huggingface/transformers/blob/b90745c5901809faef3136ed09a689e7d733526c/examples/run_squad.py



### BUT, but there are some functions that enable us to do this without going the command line route.

'You could go through this route and run your training script here via command line with the associated parameters\nand the script.'

# Using Popular Libraries

In [44]:
!pip install datasets

In [45]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

# Load the datasets
ds = load_dataset("issai/kazqad", "kazqad")
ds_1 = load_dataset("issai/kazqad", "nq-translate-kk")

# Initialize a dictionary to store merged splits
merged_splits = {}

# Iterate over available splits in ds
for split in ds.keys():
    if split in ds_1:  # If the split exists in both datasets
        # Concatenate the split from both datasets
        merged_splits[split] = concatenate_datasets([ds[split], ds_1[split]])
    else:
        # If the split only exists in ds, just use it as is
        merged_splits[split] = ds[split]

# Handle the case where ds_1 has a 'train' split not in ds
if "train" in ds_1 and "train" not in ds:
    merged_splits["train"] = ds_1["train"]

# Convert to DatasetDict for consistency
merged_dataset = DatasetDict(merged_splits)

# Optional: Display the structure of the merged dataset
print(merged_dataset)


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/issai/kazqad/resolve/main/README.md HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/issai/kazqad/resolve/2e43ff9596a8e075c3cd0ba3ab7584a74afca6bc/kazqad.py HTTP/11" 404 0
DEBUG:urllib3.connectionpool:Resetting dropped connection: s3.amazonaws.com
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/issai/kazqad/issai/kazqad.py HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/issai/kazqad/resolve/2e43ff9596a8e075c3cd0ba3ab7584a74afca6bc/.huggingface.yaml HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://datasets-server.huggingface.co:443 "GET /info?dataset=issai/kazqad HTTP/11" 200 None
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/issai/kazqad/resolve/2e43ff9596a8e075c3cd0ba3ab7584a74afca6bc/dataset_infos.json HTTP/11" 404 0
DEBUG:filelock:Attempting to acqui

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 64769
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 764
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2713
    })
})


In [46]:
squad = merged_dataset
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 64769
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 764
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2713
    })
})

In [47]:
# More specific information about the dataset can be found here: https://huggingface.co/datasets/squad#data-instances
squad["train"][0]
# id -> hash of the context
# title -> Document where the context resides
# Context -> Information where the answer resides
# Question -> What question are you trying to find the answer to?
# Answers -> What is the answer to the question? And the location on where in the text the answer begins (span)

{'id': '-9219179079952330185#682455_1_1_1',
 'title': 'Карди Би',
 'context': 'Belcalis Marlenis Almánzar ( /ˈbɛlkəliːz ɑːlˈmænzɑr/, Spanish: [alˈmansaɾ]; 1992 жылы 11 қазанда дүниеге келген), кәсіби түрде Cardi B ретінде белгілі, американдық рэпер, әнші және ән жазушы. Ол өзінің агрессивті ағынымен және ашық сөздерімен танымал. Нью-Йорк қаласында туып-өскен ол Вайн және Инстагрмда-да танымалдылыққа қол жеткізіп, интернетте атақты болды. 2015 жылдан 2017 жылға дейін ол VH1 реалити-сериалының «Махаббат және хип хоп: Нью-Йорк» сериалында тұрақты актер мүшесі ретінде пайда болды, ол оның музыкалық талпыныстарын бейнелейді; екі микстейп шығарған — Gangsta Bitch Music, Vol. 1 (2016) және том. 2 (2017), 2017 жылдың ақпанында Atlantic Records -пен қол қойылғанға дейін. Музыкадан тыс, ол 2022 жылы Playboy ойын-сауық журналының креативті директоры болды. Оның алғашқы студиялық альбомы, Invasion of Privacy (2018), Billboard 200 тізімінде бірінші орынға шықты, RIAA үштік платина сертификатын алды

In [48]:
# Preprocess the data to a BERT format
def preprocess_function(examples):
    """Courtesy of https://huggingface.co/docs/transformers/tasks/question_answering"""
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [49]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

In [50]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

# CPU Training Result... Let's think about GPU's okay?


In [51]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# model = AutoModelForQuestionAnswering.from_pretrained(model_name) # remember that model_name is deepset/roberta-base-squad2

In [52]:
!nvidia-smi


Sun Nov  3 06:05:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L40S                    On  | 00000000:04:00.0 Off |                    0 |
| N/A   39C    P0              80W / 350W |  37650MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [53]:
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=0.01,
#     per_device_train_batch_size=54,  # Increased batch size
#     per_device_eval_batch_size=54,  # Increased batch size
#     num_train_epochs=2,
#     weight_decay=0.01,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_squad["train"],
#     eval_dataset=tokenized_squad["validation"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

# trainer.train()

In [54]:
!pip install peft

In [55]:
from peft import get_peft_model, LoraConfig
from transformers import Trainer, TrainingArguments, get_scheduler

In [ ]:
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.2,
    use_dora=True
)
model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",        # Match eval strategy with save strategy
    save_strategy="steps",              # Save checkpoints every specified number of steps
    save_steps=500,                      # Save every 10 steps
    learning_rate=2e-5, 
    per_device_train_batch_size=54,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_steps=500,
    lr_scheduler_type="linear"
)


# Initialize the Trainer with a custom callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Now train
trainer.train()


/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_9757/1591475149.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
500,4.040200,No log
1000,3.071500,No log
1500,2.893700,No log
2000,2.799400,No log
2500,2.732800,No log


DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nur-dev/roberta-kaz-large/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nur-dev/roberta-kaz-large/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nur-dev/roberta-kaz-large/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nur-dev/roberta-kaz-large/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nur-dev/roberta-kaz-large/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /nur-dev/roberta-kaz-large/resolve/main/config.json HTTP/11" 200 0
DEBU

In [ ]:
save_path = "./results/final"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path) 